## Multi Input - somewhat reinforcement learning

3 inputs
1. all train input samples in one array
2. all train output samples in one array
2. the test input
3. the past predictions for this task

2 output
1. single pixel - value 0-9
2. the predicted size of the task


SUDO CODE TRAINING: 

1. get perfect samples for every pixel
2. predict every sample pixel 100.000x (e-greedy: first random then with model)
3. get discounted rewards for every prediction - (1 point for completed column, 1 point for completed row, 1 point for right pixel value)
4. train network on all collected samples
5. iterate over 2-4 until convergence ;)

In [1]:
#base
import os
import numpy as np
import pandas as pd

#tensorflow
from tensorflow.keras.layers import Input, Dense, Flatten, Conv1D, MaxPooling1D, concatenate, Subtract
from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

import numpy as np
np.random.seed(0)  # Set a random seed for reproducibility

# utils
import utils.file_handling as io
from utils import plotting as plt


In [1]:
# tensoflow-gpu test 

import tensorflow as tf

tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)


Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [47]:
# MULTI-Input-Output-CNN

def create_model():
    input_X1 = Input(shape=(None, 1))
    input_X2 = Input(shape=(None, 1))
    input_Y1 = Input(shape=(None, 1))
    input_Y2 = Input(shape=(1024, 1))
    x_1 = Conv1D(filters=32, kernel_size=(4), strides=1, padding='same')(input_X1)
    x_2 = Conv1D(filters=32, kernel_size=(4), strides=1, padding='same')(input_X2)
    x_sub = Subtract()([x_1, x_2])
    x_fin = Conv1D(32, kernel_size=(4), strides=1, padding='same', name='training_task_final_layer')(x_sub)
    
    y_1 = Conv1D(filters=32, kernel_size=(4), strides=1, padding='same')(input_Y1)
    y_1_pooling = MaxPooling1D(4)(y_1)
    #y_1_flatten = Flatten()(y_1_pooling)
    y_2 = Conv1D(filters=32, kernel_size=(4), strides=1, padding='same')(input_Y2)
    y_2_pooling = MaxPooling1D(4)(y_2)
    #y_2_flatten = Flatten()(y_2_pooling)
    y_con = concatenate([y_1_pooling, y_2_pooling])
    y_fin = Conv1D(32, kernel_size=(4), strides=1, padding='same', name='test_task_final_layer')(y_con)
    
    merge = concatenate([x_fin, y_fin])
    #flat_layer = Flatten()(merge)

    
    out_1 = Dense(11, activation='relu')(merge)
    out_1 = Dense(128, activation='relu')(out_1)
    out_1 = Dense(256, activation='relu')(out_1)
    out_1 = Dense(512, activation='relu')(out_1)
    out_1 = Dense(1, activation='softmax', name='pixel_predictor')(out_1)

    out_2 = Dense(64, activation='relu')(merge)
    out_2 = Dense(1, activation='linear', name='shape_predictor')(out_2)
    
    model = Model(inputs=[input_X1, input_X2, input_Y1, input_Y2], outputs=[out_1, out_2])
    
    opt = Adam(lr=1e-3, decay=1e-3)
    losses = {
        "pixel_predictor": "categorical_crossentropy",
        "shape_predictor": "mean_absolute_error",
    }

    model.compile(loss=losses, optimizer=opt)
    return model

#multi_cnn = create_model()

In [48]:
multi_cnn = create_model()
multi_cnn.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           [(None, None, 1)]    0                                            
__________________________________________________________________________________________________
input_28 (InputLayer)           [(None, 1024, 1)]    0                                            
__________________________________________________________________________________________________
input_25 (InputLayer)           [(None, None, 1)]    0                                            
__________________________________________________________________________________________________
input_26 (InputLayer)           [(None, None, 1)]    0                                            
____________________________________________________________________________________________

In [25]:
training_path = os.getcwd()+'/data/training/'
train_data = io.get_tasks(training_path)

def preprocess_task(task):
    input_1 = [] # all training inputs flattened and concatenated
    input_2 = [] # all training outputs flattened and concatenated
    input_3 = [] # all test inputs 
    input_4 = np.array([10 for i in range(32*32)])# Initially None, will be filled with predicted values over time
    for sample in task['train']:
        input_1 = np.append(input_1, sample['input'])
        input_2 = np.append(input_2, sample['output'])
    for sample in task['test']:
        input_3 = np.append(input_3, sample['input'])
    return input_1, input_2, input_3, input_4

input_list_1, input_list_2, input_list_3, input_list_4 = [], [], [], []
for task in train_data:
    input_1, input_2, input_3, input_4 = preprocess_task(task)
    input_list_1.append(input_1)
    input_list_2.append(input_2)
    input_list_3.append(input_3)
    input_list_4.append(input_4)
input_list_3[0]

array([7., 0., 7., 7., 0., 7., 7., 7., 0.])

In [26]:
output_shape = []
for task in train_data:
    rows = len(task['test'][0]['output'])
    cols = len(task['test'][0]['output'][0])
    output_shape.append(np.array([rows, cols]))
output_shape = np.array(output_shape)
iterations_per_task = [np.prod(task) for task in output_shape]
output_shape[0]

array([9, 9])

In [10]:
len(input_list_3)

389

In [28]:
output_pixel_all_iterations = []
for task in train_data:
    full_output = np.array(task['test'][0]['output']).flatten()
    all_iters_task = []
    for i in range(len(full_output)): # number of iterations = number of pixels in test ouput
        pixel = full_output[i]
        pixel_dummies = np.array([pixel == i for i in range(0, 11)])
        all_iters_task.append(pixel_dummies)
    output_pixel_all_iterations.append(all_iters_task)
output_pixel_all_iterations[0][0]

array([False, False, False, False, False, False, False,  True, False,
       False, False])

In [29]:
len(output_pixel_all_iterations)

389

In [81]:
# MULTI-Input-Output-CNN

def create_model():
    input_X1 = Input(shape=(None, 1))
    input_X2 = Input(shape=(None, 1))
    input_Y1 = Input(shape=(None, 1))
    input_Y2 = Input(shape=(1024, 1))
    x_1 = Conv1D(filters=32, kernel_size=(4), strides=1, padding='same')(input_X1)
    x_2 = Conv1D(filters=32, kernel_size=(4), strides=1, padding='same')(input_X2)
    x_sub = Subtract()([x_1, x_2])
    x_fin = Conv1D(32, kernel_size=(4), strides=1, padding='same', name='training_task_final_layer')(x_sub)
    
    y_1 = Conv1D(filters=32, kernel_size=(4), strides=1, padding='same')(input_Y1)
    y_1_pooling = MaxPooling1D(2)(y_1)
    #y_1_flatten = Flatten()(y_1_pooling)
    y_2 = Conv1D(filters=32, kernel_size=(4), strides=1, padding='same')(input_Y2)
    y_2_pooling = MaxPooling1D(2)(y_2)
    #y_2_flatten = Flatten()(y_2_pooling)
    y_con = concatenate([y_1_pooling, y_2_pooling])
    y_fin = Conv1D(32, kernel_size=(4), strides=1, padding='same', name='test_task_final_layer')(y_con)
    
    merge = concatenate([x_fin, y_fin])
    #flat_layer = Flatten()(merge)

    
    #out_1 = Dense(11, activation='relu')(merge)
    y_1_flatten = Flatten()(merge)
    out_1 = Dense(128, activation='relu')(y_1_flatten)
    out_1 = Dense(256, activation='relu')(out_1)
    out_1 = Dense(512, activation='relu')(out_1)
    out_1 = Dense(11, activation='softmax', name='pixel_predictor')(out_1)

    #out_2 = Dense(64, activation='relu')(merge)
    #y_1_flatten = Flatten()(merge)
    #out_2 = Dense(128, activation='relu')(y_1_flatten)
    #out_2 = Dense(1, activation='linear', name='shape_predictor')(out_2)
    
    model = Model(inputs=[input_X1, input_X2, input_Y1, input_Y2], outputs=[out_1])
    
    opt = Adam(lr=1e-3, decay=1e-3)
    losses = {
        "pixel_predictor": "categorical_crossentropy"
        #"shape_predictor": "mean_absolute_error",
    }

    model.compile(loss=losses, optimizer=opt)
    return model

multi_cnn = create_model()
multi_cnn.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_63 (InputLayer)           [(None, None, 1)]    0                                            
__________________________________________________________________________________________________
input_64 (InputLayer)           [(None, 1024, 1)]    0                                            
__________________________________________________________________________________________________
input_61 (InputLayer)           [(None, None, 1)]    0                                            
__________________________________________________________________________________________________
input_62 (InputLayer)           [(None, None, 1)]    0                                            
___________________________________________________________________________________________

In [82]:
np.expand_dims(output_shape[0], axis=0)

array([[9, 9]])

In [99]:
multi_cnn.fit([np.expand_dims(input_list_1[0], 0),
        np.expand_dims(input_list_2[0], 0),
        np.expand_dims(input_list_3[0], 0),
        np.expand_dims(input_list_4[0], 0)], 
        [np.expand_dims(output_pixel_all_iterations[0][0], 0)])

ValueError: Error when checking input: expected input_61 to have 3 dimensions, but got array with shape (1, 45)

In [96]:
for i in [np.expand_dims(input_list_1[0], 0),
        np.expand_dims(input_list_2[0], 0),
        np.expand_dims(input_list_3[0], 0),
        np.expand_dims(input_list_4[0], 0),
        np.expand_dims(output_pixel_all_iterations[0][0], 0),
        np.expand_dims((output_shape[0][0]), 0)]:
    print(i.shape)

(1, 45)
(1, 405)
(1, 9)
(1, 1024)
(1, 11)
(1,)


In [97]:
np.expand_dims((output_shape[0][0]), 0)

array([9])

In [51]:
np.expand_dims(output_pixel_all_iterations[0][0], axis=0).shape

(1, 11)